# 🧠 Agent 수업 1주차 – Agent Full-Loop & 설계 심화

## 📘 개요

이 노트북은 프롬프트 엔지니어링, RAG, API 사용 경험이 있는 수강생을 위한 1주차 심화 실습 자료입니다.

### 학습 목표
1.  **Agent의 필요성**을 일반 LLM과의 비교를 통해 체감합니다.
2.  Function Calling을 이용해 <b>Agent의 전체 동작 사이클(Full-Loop)</b>을 수동으로 구현합니다.
3.  여러 도구가 있을 때 LLM이 어떻게 판단하는지 실험합니다.
4.  **Agent가 잘못된 판단(Hallucination)을 내리는 경우**를 직접 확인하고, 그에 대한 대응 설계의 필요성을 이해합니다.
5.  도구 실행 시 발생할 수 있는 에러를 어떻게 처리하는지 학습합니다.
6.  수동 Agent 구현의 한계를 통해 LangChain과 같은 프레임워크의 필요성을 체감합니다.

In [ ]:
import openai
import os
import json
import requests
import wikipedia
from dotenv import load_dotenv

load_dotenv()

client = openai.OpenAI()
MODEL = "gpt-4o-mini"

---

## 🔧 2. Tool 정의: 모든 함수와 스키마 통합
실습에 사용할 모든 함수(Tool)와 그 함수의 명세(JSON Schema)를 이 섹션에서 한 번에 정의합니다.

In [ ]:
# === Tool 1: 환율 조회 함수 및 스키마 ===
def get_exchange_rate(currency: str) -> str:
    """지정된 통화(currency)와 대한민국 원(KRW) 사이의 현재 환율을 가져옵니다."""
    if not isinstance(currency, str) or len(currency) != 3:
        return json.dumps({"error": "유효하지 않은 통화 코드입니다. 3자리 ISO 코드를 사용해야 합니다."})

    url = f"https://api.exchangerate-api.com/v4/latest/{currency.upper()}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        krw_rate = data.get("rates", {}).get("KRW")
        if krw_rate:
            return json.dumps({"rate": krw_rate})
        else:
            return json.dumps({"error": f"'{currency}'에 대한 KRW 환율 정보를 찾을 수 없습니다."})
    except Exception as e:
        return json.dumps({"error": f"API 요청 중 에러 발생: {e}"})

exchange_tool_schema = {
    "type": "function",
    "function": {
        "name": "get_exchange_rate",
        "description": "특정 통화와 대한민국 원(KRW) 사이의 실시간 환율을 조회합니다.",
        "parameters": {
            "type": "object",
            "properties": {"currency": {"type": "string", "description": "환율을 조회할 기준 통화의 3자리 ISO 4217 코드 (예: USD, JPY)"}},
            "required": ["currency"]
        }
    }
}

# === Tool 2: 위키백과 검색 함수 및 스키마 ===
def search_wikipedia(query: str) -> str:
    """위키백과에서 정보를 검색하여 요약합니다."""
    try:
        wikipedia.set_lang("ko")
        return wikipedia.summary(query, sentences=2)
    except Exception as e:
        return f"위키백과 검색 중 오류 발생: {e}"

wiki_tool_schema = {
    "type": "function",
    "function": {
        "name": "search_wikipedia",
        "description": "위키백과(Wikipedia)에서 특정 주제에 대한 정보를 검색하고 요약합니다.",
        "parameters": {
            "type": "object",
            "properties": {"query": {"type": "string", "description": "위키백과에서 검색할 키워드 또는 질문"}},
            "required": ["query"]
        }
    }
}

# === 사용 가능한 함수를 매핑 ===
available_functions = {
    "get_exchange_rate": get_exchange_rate,
    "search_wikipedia": search_wikipedia
}

---

## 🔁 3. Agent Full-Loop 수동 구현 (⭐핵심⭐)
Agent가 실제로 동작하는 전 과정을 단계별로 나누어 직접 코드를 실행해봅니다.

In [ ]:
messages = [{"role": "user", "content": "100달러는 지금 얼마인가요?"}]
# messages = [{"role": "user", "content": "서강대학교는 어디에 있나요?"}]
tools = [exchange_tool_schema]

print("💬 1단계: LLM에 Tool 호출 요청")
response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")
response_message = response.choices[0].message
messages.append(response_message)

tool_calls = response_message.tool_calls
if tool_calls:
    print(" LLM이 Tool 사용을 결정했습니다.")
    print("\n 2단계: 결정된 Tool을 실제로 실행")
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        print(f"- 실행 함수: {function_name}({', '.join([f'{k}={v}' for k, v in function_args.items()])})")

        function_response = function_to_call(**function_args)
        print(f"- 실행 결과: {function_response}")

        messages.append({"tool_call_id": tool_call.id, "role": "tool", "name": function_name, "content": function_response})

    print("\n 3단계: Tool 실행 결과를 바탕으로 최종 답변 생성")
    final_response = client.chat.completions.create(model=MODEL, messages=messages)
    print("\n===== Agent의 최종 답변 =====")
    agent_answer = final_response.choices[0].message.content
    print(agent_answer)
else:
    print(" LLM이 Tool 사용을 결정하지 않았습니다.")
    final_response = client.chat.completions.create(model=MODEL, messages=messages)
    print("\n===== Agent의 최종 답변 =====")
    print(final_response.choices[0].message.content)

---

## ⚖️ 4. 비교: GPT 단독 실행 vs Agent (Agent의 필요성)
Agent 구조의 필요성을 체감하기 위해, 동일한 질문을 Tool 없이 일반 LLM에게만 물어봅니다.

In [ ]:
response_gpt_only = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "100달러는 지금 한국 원으로 얼마인가요?"}
    ]
)
gpt_only_answer = response_gpt_only.choices[0].message.content

print("===== ① GPT 단독 답변 (실시간 정보 없음) =====")
print(gpt_only_answer)

print("\n===== ② Agent 답변 (실시간 정보 활용) =====")
print(agent_answer)

> **결과 분석**: GPT 단독 실행은 실시간 정보에 접근할 수 없으므로 **환율 정보를 제공하지 못하고 회피적인 답변**을 합니다. 반면, `get_exchange_rate` Tool을 사용한 Agent는 **정확한 실시간 정보를 바탕으로 답변**합니다. 이것이 바로 외부 세계와 상호작용하는 Agent가 필요한 이유입니다.

---

## 🤹‍♂️ 5. 심화: Multi-Tool 환경에서의 선택 (Parallel Function Calling)
LLM에게 여러 개의 도구를 제공하고, 상황에 맞는 도구를 동시에 선택하게 만들어 봅니다.

In [ ]:
# 5. 심화: Multi-Tool 환경에서의 선택 (수정된 코드)

multi_tools = [exchange_tool_schema, wiki_tool_schema]
prompt = "OpenAI는 어떤 회사인가요? 그리고 현재 1유로(EUR)는 얼마인지도 알려주세요."

# 대화 기록 초기화
messages = [{"role": "user", "content": prompt}]

# --- 1단계: LLM에 Tool 호출 요청 ---
print("===== 1단계: Multi-Tool 환경에서 LLM의 Tool 호출 결정 =====")
response_1 = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=multi_tools,
    tool_choice="auto"
)
response_message = response_1.choices[0].message

# LLM의 tool_calls 결정을 출력
print("LLM의 Tool 선택:")
print(response_message.tool_calls)

# 대화 기록에 LLM의 응답 추가
messages.append(response_message)

# --- 2단계: 결정된 모든 Tool을 실제로 실행 ---
tool_calls = response_message.tool_calls
if tool_calls:
    print("\n===== 2단계: 결정된 모든 Tool 실행 =====")
    # 루프를 돌며 모델이 요청한 모든 함수를 실행
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)

        print(f"- 실행 함수: {function_name}")

        # 함수 실행
        function_response = function_to_call(**function_args)

        # 함수 실행 결과를 대화 기록에 추가
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )

# --- 3단계: Tool 실행 결과를 바탕으로 최종 답변 생성 ---
print("\n===== 3단계: Tool 실행 결과를 종합하여 최종 답변 생성 =====")
final_response = client.chat.completions.create(
    model=MODEL,
    messages=messages
)

# 최종 답변의 content 출력
print("LLM의 최종 답변:")
print(final_response.choices[0].message.content)

---

## 🤯 6. 심화: Agent의 실패 케이스 (Hallucination)
**Agent가 있다고 항상 똑똑한 건 아닙니다.** 교묘한 질문에는 엉뚱한 도구를 선택하는 '환각(Hallucination)' 현상을 보일 수 있습니다.

In [ ]:
# 6. 심화: Agent의 실패 케이스 (Hallucination) - 수정된 코드

# '원'이라는 단어와 '가치'라는 맥락으로 환각을 유도하는 질문
prompt_hallucinate = "비트코인 창시자 사토시 나카모토의 재산은 대한민국 '원'으로 얼마인가요?"

# 대화 기록 초기화
messages = [{"role": "user", "content": prompt_hallucinate}]
tools = [exchange_tool_schema, wiki_tool_schema]

# --- 1단계: LLM에 Tool 호출 요청 ---
print(f"===== 질문: '{prompt_hallucinate}' =====")
response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=tools,
    tool_choice="auto"
)
response_message = response.choices[0].message
messages.append(response_message)

tool_calls = response_message.tool_calls
if tool_calls:
    print("\n===== 1단계 결과: LLM의 Tool 선택 =====")
    print("LLM이 잘못된 Tool을 선택했습니다 (Hallucination 발생!):")
    print(tool_calls)

    # --- 2단계: 잘못 선택된 Tool을 실제로 실행 ---
    print("\n===== 2단계: 잘못 선택된 Tool 실행 =====")
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)

        print(f"- 실행 함수: {function_name}")

        # 함수 실행 (존재하지 않는 통화이므로 에러가 담긴 결과를 반환)
        function_response = function_to_call(**function_args)
        print(f"- 실행 결과 (에러): {function_response}")

        # 함수 실행 결과를 대화 기록에 추가
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )

    # --- 3단계: 잘못된 Tool 결과를 바탕으로 최종 답변 생성 ---
    print("\n===== 3단계: 잘못된 Tool 결과를 바탕으로 생성된 최종 답변 =====")
    final_response = client.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    print(final_response.choices[0].message.content)

else:
    print("\n(이번에는 Hallucination이 발생하지 않았습니다. 모델이 올바르게 판단하여 Tool을 사용하지 않거나, 올바른 Tool을 선택했습니다.)")
    print("LLM의 답변:")
    print(response_message.content)

> **결과 분석**: '지능적인 Hallucination'과 모델의 회복 능력
이번 결과는 Agent의 동작 방식에 대해 매우 흥미로운 점을 보여줍니다.

Hallucination 발생: 예상대로, 모델은 '재산', '원'이라는 단어에 반응하여 get_exchange_rate Tool을 호출했습니다. 심지어 질문에 없던 'USD'를 스스로 가정하여 호출한 것은, 일종의 <b>'추론적 환각(Inferential Hallucination)'</b>으로 볼 수 있습니다. 여기까지만 보면 우리의 의도대로 모델이 실패한 것처럼 보입니다.

지능적인 회복: 하지만 이 모델(GPT-4-turbo)은 여기서 멈추지 않았습니다. 동시에 search_wikipedia를 호출하여 '사토시의 재산은 정확히 알 수 없다'는 핵심적인 <b>맥락(Context)</b>을 파악했습니다. 최종적으로, 두 Tool의 결과를 종합하여 '정확한 액수는 알 수 없지만, 만약 계산하고 싶다면 이런 단계를 따르면 된다'는 매우 지능적이고 유용한 '해결 계획'을 제시하며 스스로의 추론적 오류를 이겨냈습니다.

핵심 교훈: 이 결과는 매우 중요한 점을 시사합니다. 최상위 모델은 이처럼 스스로의 작은 실수를 회복하는 능력을 갖추고 있지만, 성능이 조금 더 낮은 다른 LLM을 사용했다면 잘못된 환율 정보만으로 엉뚱한 최종 답변을 생성했을 가능성이 높습니다.

따라서 어떤 LLM을 사용하든, Agent 시스템을 설계할 때는 모델이 항상 완벽할 것이라 가정하지 않고, 이처럼 예상치 못한 Tool 사용에 대비하는 안전장치(Guardrails)를 마련하는 것이 중요합니다.

---

## 🛡️ 7. 심화: Tool 에러 핸들링
만약 도구 실행 중에 에러가 발생하면 Agent는 어떻게 대처해야 할까요?

In [ ]:
messages = [{"role": "user", "content": "가짜 통화인 ABC 1개는 몇 원이야?"}]
tools = [exchange_tool_schema]

print("💬 1단계: LLM에 Tool 호출 요청")
response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")
response_message = response.choices[0].message
messages.append(response_message)

tool_calls = response_message.tool_calls
if tool_calls:
    print(" LLM이 Tool 사용을 결정했습니다.")
    print("\n 2단계: Tool 실행 (하지만 내부에서 에러 발생!)")
    tool_call = tool_calls[0]
    function_args = json.loads(tool_call.function.arguments)
    function_response_error = get_exchange_rate(**function_args)
    print(f"- 실행 결과(에러): {function_response_error}")

    messages.append({"tool_call_id": tool_call.id, "role": "tool", "name": "get_exchange_rate", "content": function_response_error})

    print("\n🗣️ 3단계: 에러 상황을 전달하고 최종 답변 받기")
    final_response_error = client.chat.completions.create(model=MODEL, messages=messages)

    print("\n===== 에러 상황을 보고받은 Agent의 최종 답변 =====")
    print(final_response_error.choices[0].message.content)
else:
    print(" LLM이 'ABC'가 유효하지 않은 통화임을 인지하고 Tool을 호출하지 않았습니다.")

---

## 🤔 8. 토론: Agent의 한계와 LangChain의 필요성

지금까지 우리는 Agent의 핵심 로직을 '수동'으로 한 단계씩 구현했습니다. 하지만 더 복잡한 문제는 어떨까요?

> **질문**: "미국 초대 대통령은 누구인지 위키백과에서 찾고, 그가 태어난 도시의 현재 날씨를 알려줘."

이처럼 여러 단계를 거치고, 중간 결과를 저장하며, 다음 행동을 계획하는 <b>'상태 관리(State Management)'와 '실행 루프(Execution Loop)'</b>가 필요합니다. 이것을 직접 구현하는 것은 매우 번거롭습니다.

**이러한 복잡성을 해결해주는 것이 바로 LangChain의 `AgentExecutor`입니다.**

다음 주차에는 오늘 수동으로 구현한 이 모든 과정을 LangChain이 어떻게 자동화하고 단순화하는지 배우게 됩니다.